# Data preparation

This notebook contains code that is used to prepare the MIMIC ICU cohort before starting the standardisation process

## Get DB Connection

In [2]:
import os
import psycopg2


def getConnection():
    # Connect to postgres with a copy of the MIMIC-III database
    con = psycopg2.connect(
        dbname=os.environ['POSTGRES_DB_NAME'],
        user=os.environ['POSTGRES_USER_NAME'],
        host=os.environ['POSTGRES_HOSTNAME'],
        port=os.environ['POSTGRES_PORT_NUMBER'],
        password=os.environ['POSTGRES_PASSWORD']
        )
    return con

## Create a Cohort

In [3]:
con = getConnection()


dropQuery = """drop table if exists omop_test_20220817.cohort_micro cascade"""
createQuery = '''
    create table omop_test_20220817.cohort_micro AS
        with stg1 as
    (
        select
        subject_id
        , min(chartdate) as chartdate
        , min(charttime) as charttime
        from
        mimiciv.microbiologyevents
        where
        spec_type_desc = 'BLOOD CULTURE'
        and org_itemid != 90760
        and org_name is not null
        group by subject_id
    ),
    stg2 as (
        select
        per.person_id
        , coalesce(stg1.charttime, stg1.chartdate) as anchor_time
        from stg1
        inner join mimiciv.patients pat
        on stg1.subject_id = pat.subject_id
        inner join __schema_name__.person per
        on per.person_source_value::int = pat.subject_id
        where (floor(date_part('day', stg1.chartdate - make_timestamp(pat.anchor_year, 1, 1, 0, 0, 0))/365.0) + pat.anchor_age) > 18
    )
    select
    *
    from
    stg2
'''

with con:
    with con.cursor() as cursor:
        cursor.execute(dropQuery)
        cursor.execute(createQuery)


In [5]:
query = '''select count(*) from omop_test_20220817.cohort_icd'''

import pandas as pd

df = pd.read_sql_query(query, con)
df

/tmp/ipykernel_133384/692151257.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, con)


,count
0,12240
